bad: 
1. money stat group by coin
2. money stat group by mcc
3. money stat group by bank
4. money stat group by shop
5. money stat group by city
6. card_num (overfit on bank and card)
7. money stat group by card_num (overfit on bank and card)
8. nation dominance coin type (two folds +0.001)
9. country dominance coin type

base: 0.5293031765414656

good: 
1. hour
2. money stat group by acquirer (+0.003)


In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [6]:
import sys
sys.path.append('..')

from Utils.Feature import FeatureEngineer
from Utils.CrossValidate import CrossValidate, lgb_f1_score
from Utils.read_data import read
# from Utils.plot import plot_dist_diff, plot_high_fraud, plot_high_countfraud

from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import precision_recall_curve, f1_score
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
import gc
import random
warnings.simplefilter('ignore')

import lightgbm as lgb
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, Pool
# import category_encoders as ce

TRAIN_SHAPE = 1521787
li = ['acquirer', 'bank', 'card', 'money', 'trade_cat', 'coin', 'online', 'trade_type',\
      'fallback', '3ds', 'fraud_ind', 'pay_type', 'install', 'term', 'date', 'time', 'mcc', 'shop', 'excess',\
      'city', 'nation', 'status', 'txkey']

In [22]:
# combine = read('../data/combine_mr.pkl')
train = read('../data/train_mr.pkl')
test = read('../data/test_mr.pkl')

combine = pd.concat([train, test])
combine = combine.reset_index(drop=True)
combine = combine[li]
combine = combine.reset_index(drop=True)
combine.tail()

,acquirer,bank,card,money,trade_cat,coin,online,trade_type,fallback,3ds,...,term,date,time,mcc,shop,excess,city,nation,status,txkey
1943447,6189,134381,122457,199.860001,2,62,0,2,0.0,0.0,...,0,120,235852.0,264,92742,0,5817,102,0,1403553
1943448,6769,93721,190014,543.510010,5,62,0,5,0.0,0.0,...,0,120,235920.0,291,78362,0,5817,102,0,303084
1943449,6215,54471,153937,655.510010,5,62,1,2,0.0,0.0,...,0,120,235922.0,289,54828,0,3460,46,0,1954395
1943450,6215,54471,153937,490.679993,5,62,1,2,0.0,0.0,...,0,120,235925.0,289,54828,0,3460,46,0,1954396
1943451,3348,80789,106773,424.660004,5,62,1,8,0.0,0.0,...,0,120,235945.0,380,54238,0,6580,38,0,826728


In [38]:
not_train = ['txkey', 'date', 'time', 'fraud_ind']
need_encode = ['acquirer', 'bank', 'card', 'coin', 'mcc', 'shop', 'city', 'nation']
cat = ['status', 'trade_cat', 'pay_type', 'trade_type']

In [40]:
# # count
# for col in need_encode:
#     combine[col + '_count'] = combine[col].map(combine[col].value_counts())

# agg
fe = FeatureEngineer()
agg = ['acquirer', 'bank', 'card', 'coin', 'mcc', 'shop', 'city', 'nation', 'status', 'trade_cat', 'pay_type', 'trade_type']
print('num agg...')
for col in need_encode:
    li_temp = [x for x in agg if x != col]
    for b in li_temp:
        if(((col == 'city') and (b == 'nation')) or ((col == 'card') and (b == 'bank'))):
            pass
        else:
            print(f'one {col} has how many {b}')
            fe.num_agg(combine, col, b)

# trade ratio
print('trade ratio...')
for col in need_encode:
    print(col, end=',')
    li_temp = [x for x in agg if x != col]
    for b in li_temp:
        fe.trade_ratio(combine, col, b)
        
# count by day
for i in np.linspace(0, 119, 120):
    print(i, end=',')
    for col in need_encode:
        combine.loc[(combine['date'] == i + 1), col + '_count_1'] = \
            combine[col].map(\
                combine.loc[(combine['date'] == i + 1), col].value_counts())


num agg...
one acquirer has how many bank
one acquirer has how many card
one acquirer has how many coin
one acquirer has how many mcc
one acquirer has how many shop
one acquirer has how many city
one acquirer has how many nation
one acquirer has how many status
one acquirer has how many trade_cat
one acquirer has how many pay_type
one acquirer has how many trade_type
one bank has how many acquirer
one bank has how many card
one bank has how many coin
one bank has how many mcc
one bank has how many shop
one bank has how many city
one bank has how many nation
one bank has how many status
one bank has how many trade_cat
one bank has how many pay_type
one bank has how many trade_type
one card has how many acquirer
one card has how many coin
one card has how many mcc
one card has how many shop
one card has how many city
one card has how many nation
one card has how many status
one card has how many trade_cat
one card has how many pay_type
one card has how many trade_type
one coin has how ma

In [41]:
pd.set_option('display.max_columns', None)
combine.head()

,acquirer,bank,card,money,trade_cat,coin,online,trade_type,fallback,3ds,fraud_ind,pay_type,install,term,date,time,mcc,shop,excess,city,nation,status,txkey,bank_num/acquirer,card_num/acquirer,coin_num/acquirer,mcc_num/acquirer,shop_num/acquirer,city_num/acquirer,nation_num/acquirer,status_num/acquirer,trade_cat_num/acquirer,pay_type_num/acquirer,trade_type_num/acquirer,acquirer_num/bank,card_num/bank,coin_num/bank,mcc_num/bank,shop_num/bank,city_num/bank,nation_num/bank,status_num/bank,trade_cat_num/bank,pay_type_num/bank,trade_type_num/bank,acquirer_num/card,coin_num/card,mcc_num/card,shop_num/card,city_num/card,nation_num/card,status_num/card,trade_cat_num/card,pay_type_num/card,trade_type_num/card,acquirer_num/coin,bank_num/coin,card_num/coin,mcc_num/coin,shop_num/coin,city_num/coin,nation_num/coin,status_num/coin,trade_cat_num/coin,pay_type_num/coin,trade_type_num/coin,acquirer_num/mcc,bank_num/mcc,card_num/mcc,coin_num/mcc,shop_num/mcc,city_num/mcc,nation_num/mcc,status_num/mcc,trade_cat_num/mcc,pay_type_num/mcc,trade_type_num/mcc,acquirer_num/shop,bank_num/shop,card_num/shop,coin_num/shop,mcc_num/shop,city_num/shop,nation_num/shop,status_num/shop,trade_cat_num/shop,pay_type_num/shop,trade_type_num/shop,acquirer_num/city,bank_num/city,card_num/city,coin_num/city,mcc_num/city,shop_num/city,status_num/city,trade_cat_num/city,pay_type_num/city,trade_type_num/city,acquirer_num/nation,bank_num/nation,card_num/nation,coin_num/nation,mcc_num/nation,shop_num/nation,city_num/nation,status_num/nation,trade_cat_num/nation,pay_type_num/nation,trade_type_num/nation,bank_trade_ratio/acquirer,card_trade_ratio/acquirer,coin_trade_ratio/acquirer,mcc_trade_ratio/acquirer,shop_trade_ratio/acquirer,city_trade_ratio/acquirer,nation_trade_ratio/acquirer,status_trade_ratio/acquirer,trade_cat_trade_ratio/acquirer,pay_type_trade_ratio/acquirer,trade_type_trade_ratio/acquirer,acquirer_trade_ratio/bank,card_trade_ratio/bank,coin_trade_ratio/bank,mcc_trade_ratio/bank,shop_trade_ratio/bank,city_trade_ratio/bank,nation_trade_ratio/bank,status_trade_ratio/bank,trade_cat_trade_ratio/bank,pay_type_trade_ratio/bank,trade_type_trade_ratio/bank,acquirer_trade_ratio/card,bank_trade_ratio/card,coin_trade_ratio/card,mcc_trade_ratio/card,shop_trade_ratio/card,city_trade_ratio/card,nation_trade_ratio/card,status_trade_ratio/card,trade_cat_trade_ratio/card,pay_type_trade_ratio/card,trade_type_trade_ratio/card,acquirer_trade_ratio/coin,bank_trade_ratio/coin,card_trade_ratio/coin,mcc_trade_ratio/coin,shop_trade_ratio/coin,city_trade_ratio/coin,nation_trade_ratio/coin,status_trade_ratio/coin,trade_cat_trade_ratio/coin,pay_type_trade_ratio/coin,trade_type_trade_ratio/coin,acquirer_trade_ratio/mcc,bank_trade_ratio/mcc,card_trade_ratio/mcc,coin_trade_ratio/mcc,shop_trade_ratio/mcc,city_trade_ratio/mcc,nation_trade_ratio/mcc,status_trade_ratio/mcc,trade_cat_trade_ratio/mcc,pay_type_trade_ratio/mcc,trade_type_trade_ratio/mcc,acquirer_trade_ratio/shop,bank_trade_ratio/shop,card_trade_ratio/shop,coin_trade_ratio/shop,mcc_trade_ratio/shop,city_trade_ratio/shop,nation_trade_ratio/shop,status_trade_ratio/shop,trade_cat_trade_ratio/shop,pay_type_trade_ratio/shop,trade_type_trade_ratio/shop,acquirer_trade_ratio/city,bank_trade_ratio/city,card_trade_ratio/city,coin_trade_ratio/city,mcc_trade_ratio/city,shop_trade_ratio/city,nation_trade_ratio/city,status_trade_ratio/city,trade_cat_trade_ratio/city,pay_type_trade_ratio/city,trade_type_trade_ratio/city,acquirer_trade_ratio/nation,bank_trade_ratio/nation,card_trade_ratio/nation,coin_trade_ratio/nation,mcc_trade_ratio/nation,shop_trade_ratio/nation,city_trade_ratio/nation,status_trade_ratio/nation,trade_cat_trade_ratio/nation,pay_type_trade_ratio/nation,trade_type_trade_ratio/nation,acquirer_count_1,bank_count_1,card_count_1,coin_count_1,mcc_count_1,shop_count_1,city_count_1,nation_count_1
0,6716,12765,101514,966.580017,4,62,0,2,0.0,0.0,0.0,5,1,2,1,29.0,247,34949,0,5817,102,0,1119159,74622,84353,2,126,18121,101,1,3,5,9,6,2,2,1,3,5,2,

In [42]:
X = combine.loc[:TRAIN_SHAPE - 1, [x for x in combine.columns if x not in not_train and x not in need_encode]]
y = combine.loc[:TRAIN_SHAPE - 1, 'fraud_ind']
print(X.shape, y.shape)

(1521787, 193) (1521787,)


In [43]:
# by 0.191 threshold
cv = CrossValidate()
res = cv.expanding_window(X, y, cat, boost_round=1000)
print(sum(res) / len(res))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's f1: 0.593791
[200]	valid_0's f1: 0.632346
[300]	valid_0's f1: 0.646385
[400]	valid_0's f1: 0.651059
[500]	valid_0's f1: 0.655365
[600]	valid_0's f1: 0.657585
Early stopping, best iteration is:
[564]	valid_0's f1: 0.658353
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's f1: 0.586401
[200]	valid_0's f1: 0.616673
[300]	valid_0's f1: 0.619876
[400]	valid_0's f1: 0.625208
[500]	valid_0's f1: 0.62705
[600]	valid_0's f1: 0.629988
[700]	valid_0's f1: 0.632951
[800]	valid_0's f1: 0.634059
[900]	valid_0's f1: 0.63348
Early stopping, best iteration is:
[825]	valid_0's f1: 0.635477
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's f1: 0.493035
[200]	valid_0's f1: 0.511359
[300]	valid_0's f1: 0.514308
Early stopping, best iteration is:
[249]	valid_0's f1: 0.516581
0.6034704709308101


# count by period

In [87]:
drop_list = []
# count
# for col in need_encode:
#     drop_list.append(col + '_count')
#     combine[col + '_count'] = combine[col].map(combine[col].value_counts())
    
# # count by 30 days
# for i in [0, 30, 60]:
#     for col in need_encode:
#         combine.loc[(combine['date'] >= i + 1) & (combine['date'] <= i + 30), col + '_count_30'] = \
#             combine[col].map(\
#                 combine.loc[(combine['date'] >= i + 1) & (combine['date'] <= i + 30), col].value_counts())
        
# # count by 15 days
# for i in [0, 15, 30, 45, 60, 75]:
#     for col in need_encode:
#         combine.loc[(combine['date'] >= i + 1) & (combine['date'] <= i + 15), col + '_count_15'] = \
#             combine[col].map(\
#                 combine.loc[(combine['date'] >= i + 1) & (combine['date'] <= i + 15), col].value_counts())

# # count by 3 days
# for i in np.linspace(0, 87, 30):
#     for col in need_encode:
#         drop_list.append(col + '_count_3')
#         combine.loc[(combine['date'] >= i + 1) & (combine['date'] <= i + 3), col + '_count_3'] = \
#             combine[col].map(\
#                 combine.loc[(combine['date'] >= i + 1) & (combine['date'] <= i + 3), col].value_counts())
# count by 1 days


0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0,101.0,102.0,103.0,104.0,105.0,106.0,107.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0,115.0,116.0,117.0,118.0,119.0,

# money stat

In [45]:
fe = FeatureEngineer()
fe.numerical_stat(combine, 'money', need_encode, var=False, mean=False, min_val=False)

In [46]:
print(combine.columns.shape)

(237,)


In [47]:
X = combine.loc[:TRAIN_SHAPE - 1, [x for x in combine.columns if x not in not_train and x not in need_encode]]
y = combine.loc[:TRAIN_SHAPE - 1, 'fraud_ind']
print(X.shape, y.shape)

# by 0.191 threshold
cv = CrossValidate()
res = cv.expanding_window(X, y, cat, boost_round=1000)
print(sum(res) / len(res))

(1521787, 225) (1521787,)
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's f1: 0.598699
[200]	valid_0's f1: 0.63355
[300]	valid_0's f1: 0.64864
[400]	valid_0's f1: 0.650961
[500]	valid_0's f1: 0.656147
[600]	valid_0's f1: 0.661712
[700]	valid_0's f1: 0.662217
Early stopping, best iteration is:
[662]	valid_0's f1: 0.663911
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's f1: 0.593158
[200]	valid_0's f1: 0.621632
[300]	valid_0's f1: 0.626402
[400]	valid_0's f1: 0.628923
[500]	valid_0's f1: 0.635616
[600]	valid_0's f1: 0.641929
[700]	valid_0's f1: 0.642093
Early stopping, best iteration is:
[622]	valid_0's f1: 0.643519
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's f1: 0.493133
[200]	valid_0's f1: 0.520095
[300]	valid_0's f1: 0.523936
Early stopping, best iteration is:
[234]	valid_0's f1: 0.528462
0.6119641890944095


In [49]:
split, gain = train_submit(combine, cat, not_train + need_encode, 'fix_num_agg_bug')

(1521787, 225)
[50]	training's f1: 0.714315
[100]	training's f1: 0.76943
[150]	training's f1: 0.772702
[200]	training's f1: 0.773609
[250]	training's f1: 0.774929
[300]	training's f1: 0.775829
[350]	training's f1: 0.779789
[400]	training's f1: 0.78397
[450]	training's f1: 0.788617
[500]	training's f1: 0.792991
[550]	training's f1: 0.798607
[600]	training's f1: 0.803689
[650]	training's f1: 0.808456
[700]	training's f1: 0.81264
[750]	training's f1: 0.817832
[800]	training's f1: 0.822608
[850]	training's f1: 0.82719
[900]	training's f1: 0.831616
[950]	training's f1: 0.836088
[1000]	training's f1: 0.840089


In [108]:
train_submit(combine, cat, not_train + need_encode, 'add_money_stat')

(1521787, 227)
[50]	training's f1: 0.715998
[100]	training's f1: 0.766833
[150]	training's f1: 0.768541
[200]	training's f1: 0.769601
[250]	training's f1: 0.772177
[300]	training's f1: 0.774359
[350]	training's f1: 0.777275
[400]	training's f1: 0.781982
[450]	training's f1: 0.786125
[500]	training's f1: 0.792142
[550]	training's f1: 0.797225
[600]	training's f1: 0.802132
[650]	training's f1: 0.807199
[700]	training's f1: 0.811493
[750]	training's f1: 0.816349
[800]	training's f1: 0.820849
[850]	training's f1: 0.824618
[900]	training's f1: 0.829361
[950]	training's f1: 0.833458
[1000]	training's f1: 0.837017


# Choose threshold

In [83]:
def threshold_search(y_true, y_proba):
    precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
    thresholds = np.append(thresholds, 1.001) 
    F = 2 / (1 / precision + 1 / recall) 
    best_score = np.max(F)
    best_th = thresholds[np.argmax(F)]
    search_result = {'threshold': best_th , 'f1': best_score}
    return search_result 

def get_f1_score(threshold, y_true, y_proba):
    y_pred = np.where(y_proba >= threshold, 1, 0)
    return f1_score(y_true, y_pred)

def find_best_threshold(X, y, cat, n_fold, boost_round=1000):
    shape_fold = int(X.shape[0] / (n_fold + 1))
    shape_list = []
    for i in range(n_fold):
        shape_list.append(shape_fold * (i + 1))
    Xs = np.split(X, shape_list)
    ys = np.split(y, shape_list)
    print(shape_list)
    
    params = {
        'objective': 'binary',
        'early_stopping_rounds': 100,
        'learning_rate': 0.01,
        'reg_alpha': 0.5,
        'reg_lambda': 0.5,
        'max_depth': -1,
        'num_leaves': 100,
        'metric': 'None',
        'seed': 6
    }
    try_threshold = np.linspace(0.001, 0.999, 999)
    df = pd.DataFrame({'threshold': try_threshold})
    for i in range(n_fold):
        train_data = lgb.Dataset(data=Xs[i], label=ys[i], categorical_feature=cat)
        val_data = lgb.Dataset(data=Xs[i + 1], label=ys[i + 1], reference=train_data, categorical_feature=cat)
        clf = lgb.train(
            params, 
            train_data,
            valid_sets=[val_data],
            num_boost_round=boost_round,
            verbose_eval=100, 
            feval=lgb_f1_score)
        
        # get best result f1 for val set
        prob = clf.predict(Xs[i + 1])
        search_result = threshold_search(ys[i + 1], prob)
        
        # add result
        df['fold_' + str(i)] = df['threshold'].apply(lambda x: get_f1_score(x, ys[i + 1], prob))
        df['fold_' + str(i)] = df['fold_' + str(i)] - search_result['f1']
        print(f'{i} fold run: search threshold {search_result}')
    return df

In [84]:
df_threshold = find_best_threshold(X, y, cat, n_fold=10)

[138344, 276688, 415032, 553376, 691720, 830064, 968408, 1106752, 1245096, 1383440]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's f1: 0.436932
[200]	valid_0's f1: 0.565789
[300]	valid_0's f1: 0.596536
[400]	valid_0's f1: 0.615768
[500]	valid_0's f1: 0.622843
[600]	valid_0's f1: 0.623515
Early stopping, best iteration is:
[508]	valid_0's f1: 0.623718
0 fold run: search threshold {'threshold': 0.09488318708086685, 'f1': 0.6676904176904177}
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's f1: 0.425792
[200]	valid_0's f1: 0.590909
[300]	valid_0's f1: 0.626307
[400]	valid_0's f1: 0.640025
[500]	valid_0's f1: 0.652696
[600]	valid_0's f1: 0.657037
[700]	valid_0's f1: 0.660117
[800]	valid_0's f1: 0.663391
Early stopping, best iteration is:
[783]	valid_0's f1: 0.663391
1 fold run: search threshold {'threshold': 0.14329370490975127, 'f1': 0.7230261487687231}
Training until validation scores don't improve for 100 rounds.
[100]	valid_0

In [90]:
df_threshold['max'] = df_threshold.iloc[:, 1:].max(axis=1)
df_threshold['min'] = df_threshold.iloc[:, 1:].min(axis=1)
df_threshold['mean'] = df_threshold.iloc[:, 1:].mean(axis=1)
df_threshold['std'] = df_threshold.iloc[:, 1:].std(axis=1)
df_threshold

,threshold,fold_0,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9,max,min,mean,std
0,0.001,-0.468523,-0.413010,-0.438365,-0.306469,-0.410548,-0.492114,-0.478816,-0.415876,-0.415654,-0.448361,-0.306469,-0.492114,-0.423860,0.059951
1,0.002,-0.381243,-0.323806,-0.362861,-0.235618,-0.341520,-0.446336,-0.427433,-0.365897,-0.380327,-0.412459,-0.235618,-0.446336,-0.363288,0.068055
2,0.003,-0.329459,-0.281685,-0.315094,-0.195051,-0.301055,-0.409496,-0.392330,-0.328113,-0.357040,-0.383192,-0.195051,-0.409496,-0.324755,0.070460
3,0.004,-0.291641,-0.244453,-0.283001,-0.168192,-0.269503,-0.385115,-0.367501,-0.301774,-0.333768,-0.355663,-0.168192,-0.385115,-0.296160,0.072014
4,0.005,-0.261379,-0.219933,-0.253932,-0.147284,-0.248044,-0.366147,-0.346326,-0.276302,-0.313708,-0.331060,-0.147284,-0.366147,-0.273129,0.072638
5,0.006,-0.238682,-0.196944,-0.232577,-0.131837,-0.227760,-0.345283,-0.326573,-0.256723,-0.298917,-0.310643,-0.131837,-0.345283,-0.253588,0.071676
6,0.007,-0.223731,-0.181947,-0.217061,-0.121637,-0.210208,-0.327518,-0.308140,-0.239797,-0.289370,-0.290903,-0.121637,-0.327518,-0.238289,0.069536
7,0.008,-0.209868,-0.168870,-0.206041,-0.111296,-0.196341,-0.311934,-0.292319,-0.232010,-0.276276,-0.270639,-0.111296,-0.311934,-0.224902,0.067593
8,0.009,-0.195095,-0.154612,-0.192901,-0.102180,-0.186067,-0.301325,-0.277389,-0.217667,-0.266176,-0.257408,-0.102180,-0.301325,-0.212860,0.067076
9,0.010,-0.183452,-0.144961,-0.182474,-0.095337,-0.174702,-0.288535,-0.263805,-0.206776,-0.256553,-0.245023,-0.095337,-0.288535,-0.202124,0.065271


In [99]:
df_threshold[df_threshold['std'] == df_threshold['std'].min()]

,threshold,fold_0,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9,max,min,mean,std
190,0.191,-0.00587,-0.004928,-0.010884,-0.003573,-0.01362,-0.013641,-0.006611,-0.009139,-0.002253,-0.007156,-0.002253,-0.013641,-0.007797,0.004149


# Clean

In [36]:
del combine
gc.collect()

405

In [57]:
import pickle as pkl
with open('../data/combine_615974.pkl', 'wb') as f:
    pkl.dump(combine, f)